In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [26]:
# Parameters
SAMPLES = "Broad_mito_1,VIB_Hydrop_1,Broad_mito_2,VIB_Hydrop_2,Broad_1,Sanger_1,Broad_2,CNAG_1,Stanford_2,Stanford_1,CNAG_2,s3atac,VIB_2,VIB_1,Sanger_2,VIB_Hydrop_1,VIB_Hydrop_2"
METADATAPKL = "out_qc_merged/data/pycistopic/qc/metadata_bc_dict.pkl"
PROFDATAPKL = "out_qc_merged/data/pycistopic/qc/profile_data_dict.pkl"
WORKFLOW_PARAMETERS = "{\"container\":\"/staging/leuven/stg_00002/lcb/cflerin/containers/aertslab-pycistopic-latest.sif\",\"biomart_annot\":{\"biomart_dataset_name\":\"hsapiens_gene_ensembl\",\"biomart_host\":\"http://www.ensembl.org\"},\"macs2_call_peaks\":{\"gsize\":\"hs\",\"qvalue\":0.01,\"extsize\":146,\"shift\":73,\"keepdup\":\"all\"},\"compute_qc_stats\":{\"n_frag\":100,\"tss_flank_window\":2000,\"tss_window\":50,\"tss_minimum_signal_window\":100,\"tss_rolling_window\":10,\"min_norm\":0.1},\"call_cells\":{\"report_ipynb\":\"/src/pycistopic/bin/pycisTopic_qc_report_template.ipynb\",\"use_density_coloring_on_scatterplot\":true,\"use_detailed_title_on_scatterplot\":true,\"filter_frags_lower\":{\"Broad_1\":3000,\"Broad_2\":3000,\"Broad_mito_1\":3500,\"Broad_mito_2\":3500,\"CNAG_1\":2000,\"CNAG_2\":2500,\"Sanger_1\":7000,\"Sanger_2\":7000,\"VIB_1\":3500,\"VIB_2\":3500,\"Stanford_1\":7000,\"Stanford_2\":4000,\"atac_pbmc_5k_v1\":3000,\"atac_pbmc_5k_nextgem\":3000,\"pbmc_unsorted_3k\":2000,\"s3atac\":10000,\"VIB_Hydrop_1\":,\"VIB_Hydrop_2\":1000},\"filter_tss_lower\":{\"Broad_1\":17,\"Broad_2\":17,\"Broad_mito_1\":12,\"Broad_mito_2\":12,\"CNAG_1\":13,\"CNAG_2\":13,\"Sanger_1\":10,\"Sanger_2\":10,\"VIB_1\":16,\"VIB_2\":13,\"Stanford_1\":10,\"Stanford_2\":10,\"atac_pbmc_5k_v1\":16,\"atac_pbmc_5k_nextgem\":16,\"pbmc_unsorted_3k\":15,\"s3atac\":4,\"VIB_Hydrop_1\":15,\"VIB_Hydrop_2\":15},\"filter_frags_upper\":\"\",\"filter_tss_upper\":\"\",\"filter_frip_lower\":\"\",\"filter_frip_upper\":\"\",\"filter_dup_rate_lower\":\"\",\"filter_dup_rate_upper\":\"\"}}"
WORKFLOW_PARAMETERS = "{\"container\":\"/staging/leuven/stg_00002/lcb/cflerin/containers/aertslab-pycistopic-latest.sif\",\"biomart_annot\":{\"biomart_dataset_name\":\"hsapiens_gene_ensembl\",\"biomart_host\":\"http://www.ensembl.org\"},\"macs2_call_peaks\":{\"gsize\":\"hs\",\"qvalue\":0.01,\"extsize\":146,\"shift\":73,\"keepdup\":\"all\"},\"compute_qc_stats\":{\"n_frag\":100,\"tss_flank_window\":2000,\"tss_window\":50,\"tss_minimum_signal_window\":100,\"tss_rolling_window\":10,\"min_norm\":0.1},\"call_cells\":{\"report_ipynb\":\"/src/pycistopic/bin/pycisTopic_qc_report_template.ipynb\",\"use_density_coloring_on_scatterplot\":true,\"use_detailed_title_on_scatterplot\":true,\"filter_frags_lower\":{\"Broad_1\":3000,\"Broad_2\":3000,\"Broad_mito_1\":3500,\"Broad_mito_2\":3500,\"CNAG_1\":1500,\"CNAG_2\":2000,\"Sanger_1\":3000,\"Sanger_2\":3000,\"VIB_1\":2000,\"VIB_2\":3500,\"Stanford_1\":2000,\"Stanford_2\":2000,\"atac_pbmc_5k_v1\":3000,\"atac_pbmc_5k_nextgem\":3000,\"pbmc_unsorted_3k\":2000,\"s3atac\":1500,\"VIB_Hydrop_1\":900,\"VIB_Hydrop_2\":900},\"filter_tss_lower\":{\"Broad_1\":17,\"Broad_2\":17,\"Broad_mito_1\":12,\"Broad_mito_2\":12,\"CNAG_1\":13,\"CNAG_2\":13,\"Sanger_1\":10,\"Sanger_2\":10,\"VIB_1\":16,\"VIB_2\":13,\"Stanford_1\":10,\"Stanford_2\":10,\"atac_pbmc_5k_v1\":16,\"atac_pbmc_5k_nextgem\":16,\"pbmc_unsorted_3k\":15,\"s3atac\":4,\"VIB_Hydrop_1\":15,\"VIB_Hydrop_2\":15},\"filter_frags_upper\":\"\",\"filter_tss_upper\":\"\",\"filter_frip_lower\":\"\",\"filter_frip_upper\":\"\",\"filter_dup_rate_lower\":\"\",\"filter_dup_rate_upper\":\"\"}}"

In [27]:
import pycisTopic
pycisTopic.__version__

'0.1.dev300+g7494158'

In [28]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [29]:
import pybiomart as pbm
import pandas as pd
import pickle
import re
import os
import json

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

%matplotlib inline

In [30]:
params = json.loads(WORKFLOW_PARAMETERS)

sample_ids = SAMPLES.split(',')

print(f"SAMPLES: {sample_ids}")
print(f"pycisTopic parameters: {json.dumps(params, indent=4)}")

SAMPLES: ['Broad_mito_1', 'VIB_Hydrop_1', 'Broad_mito_2', 'VIB_Hydrop_2', 'Broad_1', 'Sanger_1', 'Broad_2', 'CNAG_1', 'Stanford_2', 'Stanford_1', 'CNAG_2', 's3atac', 'VIB_2', 'VIB_1', 'Sanger_2', 'VIB_Hydrop_1', 'VIB_Hydrop_2']
pycisTopic parameters: {
    "container": "/staging/leuven/stg_00002/lcb/cflerin/containers/aertslab-pycistopic-latest.sif",
    "biomart_annot": {
        "biomart_dataset_name": "hsapiens_gene_ensembl",
        "biomart_host": "http://www.ensembl.org"
    },
    "macs2_call_peaks": {
        "gsize": "hs",
        "qvalue": 0.01,
        "extsize": 146,
        "shift": 73,
        "keepdup": "all"
    },
    "compute_qc_stats": {
        "n_frag": 100,
        "tss_flank_window": 2000,
        "tss_window": 50,
        "tss_minimum_signal_window": 100,
        "tss_rolling_window": 10,
        "min_norm": 0.1
    },
    "call_cells": {
        "report_ipynb": "/src/pycistopic/bin/pycisTopic_qc_report_template.ipynb",
        "use_density_coloring_on_scatterpl

In [31]:
# Load metadata
infile = open(METADATAPKL, 'rb')
metadata_bc_dict = pickle.load(infile)
infile.close()

FileNotFoundError: [Errno 2] No such file or directory: 'out_qc_merged/data/pycistopic/qc/metadata_bc_dict.pkl'

In [32]:
# Load profile data
infile = open(PROFDATAPKL, 'rb')
profile_data_dict = pickle.load(infile)
infile.close()

FileNotFoundError: [Errno 2] No such file or directory: 'out_qc_merged/data/pycistopic/qc/profile_data_dict.pkl'

# make some publication quality plots

In [5]:
import seaborn as sns
from collections import OrderedDict

In [6]:
# display names of each sample. NOTE: the samples will be dislpayed in the order of this dict!
alias_dict = OrderedDict({
    "Broad_1": "BioRad ATAC 1",
    "Broad_2": "BioRad ATAC 2",
    "Stanford_1": "10x ATAC A1",
    "Stanford_2": "10x ATAC A2",
    "VIB_1": "10x ATAC B1",
    "VIB_2": "10x ATAC B2",
    "CNAG_1": "10x ATAC C1",
    "CNAG_2": "10x ATAC C2",
    "Broad_mito_1": "10x mtATAC 1",
    "Broad_mito_2": "10x mtATAC 2",
    "Sanger_1": "10x Multiome 1",
    "Sanger_2": "10x Multiome 2",
    "VIB_Hydrop_1": "Hydrop ATAC 1",
    "VIB_Hydrop_2": "Hydrop ATAC 2",
    "s3atac": "s3 ATAC",
    # "merged": "Merged"
})

color_dict = OrderedDict({
    "Broad_mito_1": "#9467bd",
    "Broad_mito_2": "#c5b0d5",
    "CNAG_1": "#d62728",
    "CNAG_2": "#ff9896",
    "Broad_1": "#1f77b4",
    "Broad_2": "#aec7e8",
    "Sanger_1": "#8c564b",
    "Sanger_2": "#c49c94",
    "Stanford_1": "#ff7f0e",
    "Stanford_2": "#ffbb78",
    "VIB_1": "#2ca02c",
    "VIB_2": "#98df8a",
    "VIB_Hydrop_1": "#e377c2",
    "VIB_Hydrop_2": "#f7b6d2",
    "s3atac": "#7f7f7f",
    # "merged": "#111111"
})

In [7]:
tech_dict = OrderedDict({
    "Broad_1": "BioRad ATAC",
    "Broad_2": "BioRad ATAC",
    "Stanford_1": "10x ATAC A",
    "Stanford_2": "10x ATAC A",
    "VIB_1": "10x ATAC B",
    "VIB_2": "10x ATAC B",
    "CNAG_1": "10x ATAC C",
    "CNAG_2": "10x ATAC C",
    "Broad_mito_1": "10x mtATAC",
    "Broad_mito_2": "10x mtATAC",
    "Sanger_1": "10x Multiome",
    "Sanger_2": "10x Multiome",
    "VIB_Hydrop_1": "Hydrop ATAC",
    "VIB_Hydrop_2": "Hydrop ATAC",
    "s3atac": "s3 ATAC",
    # "merged": "Merged"
})

In [8]:
alias_color_dict = {alias_dict[x]:color_dict[x] for x in alias_dict.keys()}

In [9]:
df['alias'] = [alias_dict[x] for x in df['sample_id']]
df['tech'] = [tech_dict[x] for x in df['sample_id']]

In [10]:
df['alias2'] = [alias_dict[x] for x in df['sample_id']]

In [11]:
df

,Unique_nr_frag_in_regions,sample_id,cisTopic_nr_frag,seurat_cell_type,pycisTopic_leiden_10_1.0,fmx_sample,Log_unique_nr_frag,pycisTopic_leiden_10_0.8,Unique_nr_frag,consensus_cell_type,...,cisTopic_log_nr_acc,barcode,pycisTopic_leiden_10_1.2,UMAP_harmony_1,UMAP_harmony_2,tSNE_harmony_1,tSNE_harmony_2,alias,tech,alias2
CCTCCTTCTTCATCCAGAGAG_TAAGAGGGTGGCGCCTTGCGA_CGGTACGTGTCGCTATGAATA-Broad_1,5684,Broad_1,5923,Natural killer cell,23,sampleA,3.887955,24,7726,Natural killer cell,...,3.719000,CCTCCTTCTTCATCCAGAGAG_TAAGAGGGTGGCGCCTTGCGA_CG...,21,10.603668,14.324357,3.956767,23.460202,BioRad ATAC 1,BioRad ATAC,BioRad ATAC 1
CCTTAGGACGAGAATTATCAT_CCGCGATACCTACCAGATAGG-Broad_1,7516,Broad_1,7849,B cell,20,sampleA,4.001820,22,10042,B cell,...,3.823865,CCTTAGGACGAGAATTATCAT_CCGCGATACCTACCAGATAGG,16,7.608005,-7.571940,4.023234,57.260927,BioRad ATAC 1,BioRad ATAC,BioRad ATAC 1
ATGAATAGTGCATTGCAGTGT-Broad_1,6981,Broad_1,7326,CD4+ T cell,0,sampleA,3.978728,0,9522,CD4+ T cell,...,3.799547,ATGAATAGTGCATTGCAGTGT,0,11.724223,4.751494,-21.612140,-18.295300,BioRad ATAC 1,BioRad ATAC,BioRad ATAC 1
TGTTTAGATAGGCATAAGGTA-Broad_1,3511,Broad_1,3605,Natural killer cell,23,sampleA,3.675503,24,4737,Cytotoxic T cell,...,3.518646,TGTTTAGATAGGCATAAGGTA,21,13.573638,16.315140,-50.635160,34.662899,BioRad ATAC 1,BioRad ATAC,BioRad ATAC 1
TTACAGAGGTGTTTCCAAGCT_GGACGACAGTTTCTCTAGAGC-Broad_1,8311,Broad_1,8829,Natural killer cell,23,sampleA,4.043677,24,11058,Natural killer cell,...,3.881271,TTACAGAGGTGTTTCCAAGCT_GGACGACAGTTTCTCTAGAGC,21,10.550716,14.335300,3.108778,20.429550,BioRad ATAC 1,BioRad ATAC,BioRad ATAC 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GGTTAGTTGGTATTGCAGCTCGGACAAC-s3atac,123,s3atac,127,B cell,29,sampleA,3.184975,29,1531,CD14+ monocyte,...,2.103804,GGTTAGTTGGTATTGCAGCTCGGACAAC,31,3.645544,-6.040138,44.749706,51.181226,s3 ATAC,s3 ATAC,s3 ATAC
GATTCGGTCAGTTCTCTCCTTGACGAAT-s3atac,195,s3atac,203,B cell,29,sampleB,3.289812,29,1949,CD4+ T cell,...,2.298853,GATTCGGTCAGTTCTCTCCTTGACGAAT,31,4.013797,-6.059711,48.306403,57.061874,s3 ATAC,s3 ATAC,s3 ATAC
TGCGGCCTGGTCTCATTGCCCGGAACTG-s3atac,247,s3atac,252,CD4+ T cell,8,sampleB,3.362671,9,2305,CD4+ T cell,...,2.397940,TGCGGCCTGGTCTCATTGCCCGGAACTG,8,10.516696,4.159173,-33.278084,-28.925472,s3 ATAC,s3 ATAC,s3 ATAC
GAAGAGTATTTCTCCTCCTGGTGTCGGA-s3atac,160,s3atac,169,B cell,29,sampleB,3.262451,29,1830,CD4+ T cell,...,2.220108,GAAGAGTATTTCTCCTCCTGGTGTCGGA,31,6.498169,-8.294133,4.903904,40.968488,s3 ATAC,s3 ATAC,s3 ATAC


In [15]:
import numpy as np

In [14]:

def histogram(array, nbins=100):
    """
    Draw histogram from distribution and identify centers.
    Parameters
    ---------
    array: `class::np.array`
            Scores distribution
    nbins: int
            Number of bins to use in the histogram
    Return
    ---------
    float
            Histogram values and bin centers.
    """
    array = array.ravel().flatten()
    hist, bin_edges = np.histogram(array, bins=nbins, range=None)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2.
    return hist, bin_centers


def threshold_otsu(array, nbins=100):
    """
    Apply Otsu threshold on topic-region distributions [Otsu, 1979].
    Parameters
    ---------
    array: `class::np.array`
            Array containing the region values for the topic to be binarized.
    nbins: int
            Number of bins to use in the binarization histogram
    Return
    ---------
    float
            Binarization threshold.
    Reference
    ---------
    Otsu, N., 1979. A threshold selection method from gray-level histograms. IEEE transactions on systems, man, and
    cybernetics, 9(1), pp.62-66.
    """
    hist, bin_centers = histogram(array, nbins)
    hist = hist.astype(float)
    # Class probabilities for all possible thresholds
    weight1 = np.cumsum(hist)
    weight2 = np.cumsum(hist[::-1])[::-1]
    # Class means for all possible thresholds
    mean1 = np.cumsum(hist * bin_centers) / weight1
    mean2 = (np.cumsum((hist * bin_centers)[::-1]) / weight2[::-1])[::-1]
    # Clip ends to align class 1 and class 2 variables:
    # The last value of ``weight1``/``mean1`` should pair with zero values in
    # ``weight2``/``mean2``, which do not exist.
    variance12 = weight1[:-1] * weight2[1:] * (mean1[:-1] - mean2[1:]) ** 2
    idx = np.argmax(variance12)
    threshold = bin_centers[:-1][idx]
    return threshold

In [19]:
arr = np.array(df['Unique_nr_frag'])

In [20]:
threshold_otsu(arr)

184874.05

In [21]:
hist, bin_centers = histogram(arr)

In [22]:
hist

array([25895, 15663,  2685,   604,   162,    72,    35,    22,    10,
          13,     6,     7,     6,     1,     4,     0,     4,     2,
           1,     2,     1,     1,     1,     0,     0,     2,     3,
           1,     1,     0,     1,     0,     0,     0,     2,     2,
           0,     2,     1,     2,     2,     0,     0,     1,     0,
           2,     0,     1,     0,     1,     0,     0,     0,     1,
           2,     1,     0,     0,     2,     2,     0,     0,     0,
           0,     1,     2,     0,     0,     0,     0,     0,     0,
           1,     1,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           1])

In [23]:
bin_centers

array([  4989.29,  13165.87,  21342.45,  29519.03,  37695.61,  45872.19,
        54048.77,  62225.35,  70401.93,  78578.51,  86755.09,  94931.67,
       103108.25, 111284.83, 119461.41, 127637.99, 135814.57, 143991.15,
       152167.73, 160344.31, 168520.89, 176697.47, 184874.05, 193050.63,
       201227.21, 209403.79, 217580.37, 225756.95, 233933.53, 242110.11,
       250286.69, 258463.27, 266639.85, 274816.43, 282993.01, 291169.59,
       299346.17, 307522.75, 315699.33, 323875.91, 332052.49, 340229.07,
       348405.65, 356582.23, 364758.81, 372935.39, 381111.97, 389288.55,
       397465.13, 405641.71, 413818.29, 421994.87, 430171.45, 438348.03,
       446524.61, 454701.19, 462877.77, 471054.35, 479230.93, 487407.51,
       495584.09, 503760.67, 511937.25, 520113.83, 528290.41, 536466.99,
       544643.57, 552820.15, 560996.73, 569173.31, 577349.89, 585526.47,
       593703.05, 601879.63, 610056.21, 618232.79, 626409.37, 634585.95,
       642762.53, 650939.11, 659115.69, 667292.27, 

In [24]:
arr

array([ 7726, 10042,  9522, ...,  2305,  1830,  1767])

In [25]:
arr_sorted = sorted(arr)

[901,
 901,
 901,
 902,
 902,
 902,
 902,
 903,
 903,
 904,
 905,
 905,
 907,
 908,
 908,
 908,
 909,
 910,
 911,
 911,
 911,
 911,
 913,
 913,
 914,
 914,
 914,
 914,
 915,
 915,
 915,
 916,
 916,
 916,
 916,
 917,
 917,
 917,
 917,
 917,
 918,
 918,
 919,
 921,
 922,
 923,
 923,
 924,
 926,
 928,
 928,
 928,
 929,
 929,
 931,
 932,
 933,
 933,
 933,
 934,
 934,
 935,
 935,
 935,
 936,
 936,
 938,
 938,
 938,
 938,
 939,
 939,
 939,
 939,
 939,
 940,
 940,
 942,
 943,
 943,
 943,
 944,
 944,
 945,
 945,
 945,
 946,
 947,
 948,
 948,
 949,
 949,
 950,
 951,
 954,
 955,
 955,
 955,
 955,
 956,
 958,
 959,
 960,
 961,
 961,
 961,
 962,
 962,
 964,
 964,
 964,
 965,
 965,
 966,
 966,
 967,
 968,
 968,
 968,
 969,
 970,
 970,
 970,
 971,
 971,
 972,
 972,
 972,
 972,
 973,
 974,
 974,
 975,
 976,
 976,
 977,
 978,
 978,
 978,
 979,
 980,
 982,
 982,
 983,
 983,
 984,
 984,
 985,
 985,
 986,
 987,
 989,
 990,
 991,
 992,
 992,
 994,
 995,
 995,
 995,
 996,
 996,
 997,
 997,
 998,
 998,
 999

In [ ]:
# remake scatterplots